In [65]:

import pandas as pd
    
import cx_Oracle

f=open("pwds.txt","r")
lines=f.readlines()
usr=lines[0]
pw=lines[1]
f.close()
path = 'C:\ORACLE\instantclient_19_11'

host = 'ALASTRIA'
port = 1524
service = 'CW1T2'
try:
    cx_Oracle.init_oracle_client(lib_dir= path) # use this is service isn't initialised
except Exception as err:
    print('initialised already')

cx_Oracle.clientversion()  
dsn_tns = cx_Oracle.makedsn(host, port, service_name=service)

conn = cx_Oracle.connect(user=usr, password=pw, dsn=dsn_tns)

c = conn.cursor()

query = """select distinct "WBC_CASE_D"."CASE_NUM" "Case_Number", "WBC_ES_STAGE_OF_SERVICE_D"."IS_ACTIVE_NAME" "Is_Active", "WBC_ES_STAGE_OF_SERVICE_D"."STAGE_OF_SERVICE_CD" "c3", "WBC_ES_STAGE_OF_SERVICE_D"."STAGE_OF_SERVICE_NAME" "Stage_of_Employment_Service", "WBC_DATE_D_Start"."CALENDAR_DATE" "Start_Date"
 from "CDW"."WBC_CASE_D" "WBC_CASE_D", "CDW"."WBC_ES_STAGE_OF_SERVICE_D" "WBC_ES_STAGE_OF_SERVICE_D", "CDW"."WBC_DATE_D" "WBC_DATE_D_Start", "CDW"."WBC_CASE_F" "coguda30", "CDW"."WBC_OFFICE_D" "coguda31"
 where "coguda30"."OFFICE_ORG_WID"="coguda31"."OFFICE_ORG_WID" and "coguda30"."CASE_WID"="WBC_CASE_D"."CASE_WID" and "coguda30"."ES_STAGE_OF_SERVICE_WID"="WBC_ES_STAGE_OF_SERVICE_D"."ES_STAGE_OF_SERVICE_WID" and "WBC_DATE_D_Start"."DATE_WID"="coguda30"."PLAN_START_DT_WID"

"""
q2 = """

select "WBC_CASE_D"."CASE_NUM" "Case_Number", avg("coguda10"."WEEKS_TO_ACHV_SUST_EMP_ELAPSED") "c2", "WBC_DATE_D_PlanStart"."DAY_DT" "Start_Day_Date", avg("coguda10"."WEEKS_TO_ACHV_SUST_EMP_IN_PROG") "c4", "WBC_ES_STAGE_OF_SERVICE_D"."IS_ACTIVE_NAME" "Is_Active", "WBC_ES_STAGE_OF_SERVICE_D"."STAGE_OF_SERVICE_NAME" "Stage_of_Employment_Service"
 from "CDW"."WBC_CASE_D" "WBC_CASE_D", "CDW"."WBC_CASE_F" "coguda10", "CDW"."WBC_OFFICE_D" "coguda11", "CDW"."WBC_ES_STAGE_OF_SERVICE_D" "WBC_ES_STAGE_OF_SERVICE_D", "CDW"."WBC_DATE_D" "WBC_DATE_D_PlanStart"
 where "coguda10"."OFFICE_ORG_WID"="coguda11"."OFFICE_ORG_WID" and "WBC_DATE_D_PlanStart"."FISCAL_YEAR">=2019 and "WBC_CASE_D"."CASE_TYPE_CD" in ('Employment Services') and "coguda10"."CASE_TYPE_WID"=1 and "coguda10"."SNAPSHOT_DT_WID"=(
select snapshot_dt_wid
 from cdw.wbc_run_ctl_d) and "coguda10"."CASE_WID"="WBC_CASE_D"."CASE_WID" and "coguda10"."ES_STAGE_OF_SERVICE_WID"="WBC_ES_STAGE_OF_SERVICE_D"."ES_STAGE_OF_SERVICE_WID" and "WBC_DATE_D_PlanStart"."DATE_WID"="coguda10"."PLAN_START_DT_WID"
 group by "WBC_CASE_D"."CASE_NUM", "WBC_ES_STAGE_OF_SERVICE_D"."IS_ACTIVE_NAME", "WBC_DATE_D_PlanStart"."DAY_DT", "WBC_ES_STAGE_OF_SERVICE_D"."STAGE_OF_SERVICE_NAME"



"""
result = pd.read_sql(q2, con=conn)

print(result)

initialised already
           Case_Number    c2 Start_Day_Date    c4              Is_Active  \
0      1-421153-103719  21.0     2019-01-17  21.0  Not Active In Program   
1      1-421153-103720   4.0     2018-10-02   4.0  Not Active In Program   
2      1-421153-103725   NaN     2019-02-19   NaN  Not Active In Program   
3      1-421153-103898  25.0     2019-01-09  25.0  Not Active In Program   
4      1-421153-103904   3.0     2019-07-30   3.0  Not Active In Program   
...                ...   ...            ...   ...                    ...   
97173    1-65439714237   NaN     2021-10-04   NaN      Active In Program   
97174    1-65462045764   NaN     2021-10-05   NaN      Active In Program   
97175    1-65463277560   NaN     2021-10-05   NaN      Active In Program   
97176    1-65463637612   NaN     2021-10-05   NaN      Active In Program   
97177    1-65465793162   NaN     2021-10-05   NaN      Active In Program   

                  Stage_of_Employment_Service  
0          Left Bef

In [66]:
result.nunique()
result.sort_values(by = 'Case_Number').head

<bound method NDFrame.head of            Case_Number     c2 Start_Day_Date     c4              Is_Active  \
24972  1-421153-100011  141.0     2018-08-29  128.0      Active In Program   
12793  1-421153-100014    4.0     2019-01-31    4.0  Not Active In Program   
37208  1-421153-100015   69.0     2018-09-25   69.0      Active In Program   
61397  1-421153-100050  107.0     2019-03-22  107.0      Active In Program   
73465  1-421153-100116    NaN     2019-07-08    NaN  Not Active In Program   
...                ...    ...            ...    ...                    ...   
12045    1-65465109668    NaN     2021-10-05    NaN      Active In Program   
84922    1-65465465770    NaN     2021-10-05    NaN      Active In Program   
97177    1-65465793162    NaN     2021-10-05    NaN      Active In Program   
48445    1-65465793825    NaN     2021-10-05    NaN      Active In Program   
48446    1-65466433686    NaN     2021-10-05    NaN      Active In Program   

                  Stage_of_Employ

In [69]:
k = result['Is_Active'] + ' ' + result['Stage_of_Employment_Service']
print(k.value_counts())
result.rename(columns={'Start_Day_Date':'Start_Date'},inplace = True)

Not Active In Program Left Before Outcome                   32169
Active In Program Before Sustainable Employment Achieved    23782
Not Active In Program Left After Completing Sustainment     19557
Active In Program In Sustainment                            16592
Not Active In Program Left Before Completing Sustainment     5078
dtype: int64


In [70]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
result['sus'] = None

for index, row in result.iterrows():
    if row['Is_Active'] == 'Active In Program' and row['Stage_of_Employment_Service'] == 'In Sustainment':
        result.loc[index, 'sus'] = 1
    elif row['Is_Active'] == 'Active In Program' and row['Stage_of_Employment_Service'] == 'Before Sustainable Employment Achieved':
        yr_ago = datetime.now() - relativedelta(years=2)
        if row['Start_Date'] < yr_ago:
            result.loc[index, 'sus'] = 0
        else:
            result.loc[index, 'sus'] = None
    elif row['Is_Active'] == 'Not Active In Program' and row['Stage_of_Employment_Service'] == 'Left Before Outcome':
        result.loc[index, 'sus'] = 0
    elif row['Is_Active'] == 'Not Active In Program' and row['Stage_of_Employment_Service'] == 'Left After Completing Sustainment':
        result.loc[index, 'sus'] = 1
    elif row['Is_Active'] == 'Not Active In Program' and row['Stage_of_Employment_Service'] == 'Left Before Completing Sustainment':
        result.loc[index, 'sus'] = 1
    elif row['Is_Active'] == 'Not Available':
        result.loc[index, 'sus'] = None

In [71]:
print(result['sus'].value_counts())
print(result.shape)

1    41227
0    37065
Name: sus, dtype: int64
(97178, 7)


In [72]:
result.nunique()

Case_Number                    97178
c2                               177
Start_Date                      1159
c4                               172
Is_Active                          2
Stage_of_Employment_Service        5
sus                                2
dtype: int64

In [73]:
result.to_csv('preprocessing/sustainment_data_w_case_num.csv')

In [43]:
import pandas as pd
import cx_Oracle
f=open("pwds.txt","r")
lines=f.readlines()
usr=lines[0]
pw=lines[1]
f.close()
path = 'C:\ORACLE\instantclient_19_11'

host = 'ALASTRIA'
port = 1524
service = 'CW1T2'
try:
    cx_Oracle.init_oracle_client(lib_dir= path) # use this is service isn't initialised
except Exception as err:
    print('initialised already')

cx_Oracle.clientversion()  
dsn_tns = cx_Oracle.makedsn(host, port, service_name=service)

conn = cx_Oracle.connect(user=usr, password=pw, dsn=dsn_tns)

c = conn.cursor()
pers_and_emp_facts = """select distinct "WBC_CASE_D"."CASE_NUM" "Case_Number", "WBC_LATEST_ES_OES_APP_D"."MARITAL_STATUS" "Marital_Status", "WBC_LATEST_ES_OES_APP_D"."TOTAL_DEPENDENTS" "Number_of_Dependents", "WBC_LATEST_ES_OES_APP_D"."SCHOOL_LEVEL" "Education_Level", "WBC_LATEST_ES_OES_APP_D"."CURRENTLY_IN_SCHOOL" "Currently_In_School", "WBC_LATEST_ES_OES_APP_D"."CURRENTLY_IN_SCHOOL_STATUS" "c6", "WBC_LATEST_ES_OES_APP_D"."AREA_OF_STUDY" "Education_Area_of_Study", "WBC_LATEST_ES_OES_APP_D"."INDIGENOUS_TYPE" "Indigenous_Type", "WBC_LATEST_ES_OES_APP_D"."STATUS_INDIAN" "Status_Indian", "WBC_LATEST_ES_OES_APP_D"."REFUGEE_TYPE" "Refugee_Type", "WBC_LATEST_ES_OES_APP_D"."CITIZENSHIP_STATUS" "Citizenship_Status", "WBC_LATEST_ES_OES_APP_D"."EMPLOYMENT_FACTOR_AGE" "Age", "WBC_LATEST_ES_OES_APP_D"."EMPLOYMENT_FACTOR_LACK_HOUSING" "Lack_Housing", "WBC_LATEST_ES_OES_APP_D"."EMPLOYMENT_FACTOR_ADDICTION" "Addiction", "WBC_LATEST_ES_OES_APP_D"."VISIBLE_MINORITY" "Visible_Minority", "WBC_LATEST_ES_OES_APP_D"."CHILDREN_AT_HOME" "Child_at_Home", "WBC_LATEST_ES_OES_APP_D"."SINGLE_PARENT" "Single_Parent", "WBC_LATEST_ES_OES_APP_D"."DISABILITY_PHYSICAL" "Physical_Disability", "WBC_LATEST_ES_OES_APP_D"."DISABILITY_LEARNING" "Learning_Disability", "WBC_LATEST_ES_OES_APP_D"."DEVELOPMENTAL_DISABILITY" "Developmental_Disability", "WBC_LATEST_ES_OES_APP_D"."DISABILITY_VISUAL" "Visual_Disability", "WBC_LATEST_ES_OES_APP_D"."DISABILITY_HEARING" "Hearing_Disability", "WBC_LATEST_ES_OES_APP_D"."DISABILITY_MENTAL_HEALTH" "Mental_Health_Condition", "WBC_LATEST_ES_OES_APP_D"."DISABILITY_CHRONIC_ILLNESS" "Chronic_Illness", "WBC_LATEST_ES_OES_APP_D"."DISABILITY_SHORT_TERM_ILLNESS" "Short_term_Illness", "WBC_LATEST_ES_OES_APP_D"."DISABILITY_OTHER" "Other_Disability", "WBC_LATEST_ES_OES_APP_D"."EMPLOYMENT_FACTOR_LIST" "Employment_Factor_List"
 from "CDW"."WBC_CASE_D" "WBC_CASE_D", "CDW"."WBC_LATEST_ES_OES_APP_D" "WBC_LATEST_ES_OES_APP_D", "CDW"."WBC_SERVICES_SUPPORTS_PLAN_D" "WBC_SERVICES_SUPPORTS_PLAN_D", "CDW"."WBC_DATE_D" "WBC_DATE_D_Start", "CDW"."WBC_CASE_F" "coguda40", "CDW"."WBC_OFFICE_D" "coguda41"
 where  "coguda40"."OFFICE_ORG_WID"="coguda41"."OFFICE_ORG_WID" and "WBC_CASE_D"."CASE_TYPE_CD" in ('Employment Services') and "WBC_SERVICES_SUPPORTS_PLAN_D"."PLAN_NAME" in ('Employment Services') and "WBC_DATE_D_Start"."FISCAL_YEAR">=2019 and "coguda40"."CASE_WID"="WBC_CASE_D"."CASE_WID" and "coguda40"."SRV_PLAN_WID"="WBC_SERVICES_SUPPORTS_PLAN_D"."SRV_PLAN_WID" and "WBC_DATE_D_Start"."DATE_WID"="coguda40"."PLAN_START_DT_WID" and "coguda40"."TAAAPA_WID"="WBC_LATEST_ES_OES_APP_D"."TAAAPA_WID"


"""
#from the OES Personal and Employment factors
df = pd.read_sql(pers_and_emp_facts, con=conn)

for x in df.columns:
    if x!= 'Case_Number':
        print(df[x].value_counts(dropna = False))

initialised already
Single                 36634
NaN                    25033
Married                23055
I prefer not to say     1282
Widow                      1
Name: Marital_Status, dtype: int64
NaN     67007
1.0      8741
2.0      7238
3.0      2135
4.0       638
5.0       175
6.0        46
7.0        17
8.0         7
23.0        1
Name: Number_of_Dependents, dtype: int64
NaN                                       25856
Grade 12 education, GED, or equivalent    13570
Post-secondary degree                     13089
Post-secondary diploma                     7320
Some high school (grades 10-12)            7089
Some post-secondary training               6088
Post-secondary certificate                 4994
Trades Certification                       3040
Less than grade 10                         2088
Trades Apprenticeship Training             1483
I Prefer not to say                        1315
Evergreen Certificate                        73
Name: Education_Level, dtype: int64
N      

In [44]:
#ddf = df[(df['Physical_Disability'] != "Y") & (df['Physical_Disability'] != "N")]

for x in df.columns:
    if x!= 'Case_Number':
        print(df[x].value_counts(dropna = False))
        
print(df.shape)

Single                 36634
NaN                    25033
Married                23055
I prefer not to say     1282
Widow                      1
Name: Marital_Status, dtype: int64
NaN     67007
1.0      8741
2.0      7238
3.0      2135
4.0       638
5.0       175
6.0        46
7.0        17
8.0         7
23.0        1
Name: Number_of_Dependents, dtype: int64
NaN                                       25856
Grade 12 education, GED, or equivalent    13570
Post-secondary degree                     13089
Post-secondary diploma                     7320
Some high school (grades 10-12)            7089
Some post-secondary training               6088
Post-secondary certificate                 4994
Trades Certification                       3040
Less than grade 10                         2088
Trades Apprenticeship Training             1483
I Prefer not to say                        1315
Evergreen Certificate                        73
Name: Education_Level, dtype: int64
N      60149
NaN    22723
Y

In [45]:
pp = df

pp['Marital_Status'].fillna('I prefer not to say', inplace=True)

pp['Education_Level'].fillna('I Prefer not to say', inplace=True)

pp = pp.drop(columns = ['Education_Area_of_Study','Employment_Factor_List'])

pp['Indigenous_Type'].fillna('N', inplace=True)

pp['Status_Indian'].fillna('N', inplace=True)

pp['Refugee_Type'].fillna('N', inplace=True)

pp['Citizenship_Status'].fillna('N', inplace=True)

pp['Age'].fillna('N', inplace=True)

pp['Lack_Housing'].fillna('I prefer not to say', inplace=True)

pp['Addiction'].fillna('I prefer not to say', inplace=True)

pp['Visible_Minority'].fillna('I prefer not to say', inplace=True)

pp['Child_at_Home'].fillna('I prefer not to say', inplace=True)

pp['Single_Parent'].fillna('I prefer not to say', inplace=True)

pp['Physical_Disability'].fillna('N', inplace=True)

pp['Learning_Disability'].fillna('N', inplace=True)

pp['Developmental_Disability'].fillna('N', inplace=True)

pp['Visual_Disability'].fillna('N', inplace=True)

pp['Hearing_Disability'].fillna('N', inplace=True)

pp['Mental_Health_Condition'].fillna('N', inplace=True)

pp['Short_term_Illness'].fillna('N', inplace=True)

pp['Other_Disability'].fillna('N', inplace=True)

pp['Chronic_Illness'].fillna('N', inplace=True)

pp['Number_of_Dependents'].fillna(0, inplace=True)

pp['Currently_In_School'].fillna('No', inplace=True)

for x in pp.columns:
    if x!= 'Case_Number':
        print(pp[x].value_counts(dropna = False))
        
print(pp.shape)

Single                 36634
I prefer not to say    26315
Married                23055
Widow                      1
Name: Marital_Status, dtype: int64
0.0     67007
1.0      8741
2.0      7238
3.0      2135
4.0       638
5.0       175
6.0        46
7.0        17
8.0         7
23.0        1
Name: Number_of_Dependents, dtype: int64
I Prefer not to say                       27171
Grade 12 education, GED, or equivalent    13570
Post-secondary degree                     13089
Post-secondary diploma                     7320
Some high school (grades 10-12)            7089
Some post-secondary training               6088
Post-secondary certificate                 4994
Trades Certification                       3040
Less than grade 10                         2088
Trades Apprenticeship Training             1483
Evergreen Certificate                        73
Name: Education_Level, dtype: int64
N     60149
No    22723
Y      3133
Name: Currently_In_School, dtype: int64
NaN          82872
Part-time

In [74]:
import pandas as pd
df = pd.read_csv('preprocessing/sustainment_data_w_case_num.csv')

print(pp.dtypes)

n=df[['sus', 'Case_Number']]
n = n.drop_duplicates()
print(n.dtypes)
pf = pp.merge(n, on = 'Case_Number', how = 'left')
print(pp.shape)
print(pf.shape)
print(pf.nunique())
print(pf.columns)
print(pf.sort_values(by = 'Case_Number').head)


Case_Number                  object
Marital_Status               object
Number_of_Dependents        float64
Education_Level              object
Currently_In_School          object
c6                           object
Indigenous_Type              object
Status_Indian                object
Refugee_Type                 object
Citizenship_Status           object
Age                          object
Lack_Housing                 object
Addiction                    object
Visible_Minority             object
Child_at_Home                object
Single_Parent                object
Physical_Disability          object
Learning_Disability          object
Developmental_Disability     object
Visual_Disability            object
Hearing_Disability           object
Mental_Health_Condition      object
Chronic_Illness              object
Short_term_Illness           object
Other_Disability             object
dtype: object
sus            float64
Case_Number     object
dtype: object
(86005, 25)
(86005, 26)
Ca

In [75]:

import pandas as pd
import cx_Oracle
f=open("pwds.txt","r")
lines=f.readlines()
usr=lines[0]
pw=lines[1]
f.close()
path = 'C:\ORACLE\instantclient_19_11'

host = 'ALASTRIA'
port = 1524
service = 'CW1T2'
try:
    cx_Oracle.init_oracle_client(lib_dir= path) # use this is service isn't initialised
except Exception as err:
    print('initialised already')

cx_Oracle.clientversion()  
dsn_tns = cx_Oracle.makedsn(host, port, service_name=service)

conn = cx_Oracle.connect(user=usr, password=pw, dsn=dsn_tns)

c = conn.cursor()
query2 = """

select distinct "WBC_CASE_D"."CASE_NUM" "Case_Number", "WBC_LATEST_ES_OES_APP_D"."MOST_RECENT_WORK_EMPLOYER" "Employer_Name", "WBC_LATEST_ES_OES_APP_D"."MOST_RECENT_WORK_JOB_TITLE" "Work_Title", "WBC_LATEST_ES_OES_APP_D"."MOST_RECENT_WORK_INDUSTRY" "Industry", "WBC_LATEST_ES_OES_APP_D"."MOST_RECENT_WORK_START_DATE" "Start_Date", "WBC_LATEST_ES_OES_APP_D"."MOST_RECENT_WORK_END_DATE" "End_Date", "WBC_LATEST_ES_OES_APP_D"."REASON_FOR_LEAVING_EMPLOYMENT" "Reason_for_Leaving"
 from "CDW"."WBC_CASE_D" "WBC_CASE_D", "CDW"."WBC_LATEST_ES_OES_APP_D" "WBC_LATEST_ES_OES_APP_D", "CDW"."WBC_CASE_F" "coguda20", "CDW"."WBC_OFFICE_D" "coguda21"
 where "coguda20"."OFFICE_ORG_WID"="coguda21"."OFFICE_ORG_WID" and "coguda20"."CASE_WID"="WBC_CASE_D"."CASE_WID" and "coguda20"."TAAAPA_WID"="WBC_LATEST_ES_OES_APP_D"."TAAAPA_WID"


"""
#from the OES Personal and Employment factors
df = pd.read_sql(query2, con=conn)



initialised already


In [76]:
for x in df.columns:
    if x!= 'Case_Number':
        print(df[x].value_counts(dropna = False))
a = pd.DataFrame(df['Case_Number'].value_counts().rename_axis('Case_Number').reset_index(name='counts'))
print(max(a.counts))
print(df.loc[df['End_Date'].astype(str) == '2818-05-29 00:00:00'])
df['End_Date'] = df['End_Date'].astype(str)
df['End_Date'] = df['End_Date'].replace(['2818-05-29 00:00:00'], '2018-05-29 00:00:00')

df['Start_Date'] = df['Start_Date'].astype(str)
df['End_Date'] = pd.to_datetime(df['End_Date'], errors = 'coerce')
df['Start_Date'] = pd.to_datetime(df['Start_Date'], errors = 'coerce')
df['days'] = df['End_Date'] - df['Start_Date']
g = df.groupby(['Case_Number'])['days'].agg('sum')
print(g)
b = a.join(g, on = 'Case_Number', how = 'inner')
print(b)
pf = pf.merge(b, on = 'Case_Number', how = 'left')

NaN                                        84981
self employed                                216
Self Employed                                203
Self employed                                140
Walmart                                      139
                                           ...  
Big O Tires                                    1
Gayle Raphanel Personal Law Corporation        1
Kenn Borek LTD                                 1
paamed home health                             1
R.Clausin Logging Ltd                          1
Name: Employer_Name, Length: 42569, dtype: int64
NaN                                         82333
Cashier                                       693
Labourer                                      640
Server                                        520
labourer                                      358
                                            ...  
Child and Youth Mental Health Counsellor        1
insulation laborer                              1
Senior Purse

In [78]:
pf.columns

Index(['Case_Number', 'Marital_Status', 'Number_of_Dependents',
       'Education_Level', 'Currently_In_School', 'c6', 'Indigenous_Type',
       'Status_Indian', 'Refugee_Type', 'Citizenship_Status', 'Age',
       'Lack_Housing', 'Addiction', 'Visible_Minority', 'Child_at_Home',
       'Single_Parent', 'Physical_Disability', 'Learning_Disability',
       'Developmental_Disability', 'Visual_Disability', 'Hearing_Disability',
       'Mental_Health_Condition', 'Chronic_Illness', 'Short_term_Illness',
       'Other_Disability', 'sus', 'counts', 'days'],
      dtype='object')

In [79]:
pf = pf.drop('Case_Number', axis=1)

In [88]:
tst = pf
print(tst.columns)
from sklearn import preprocessing



print(tst.dtypes)
le = preprocessing.OneHotEncoder()

tst = tst[tst['sus'].notna()]

label = tst['sus']
tst = tst.drop('sus', axis = 1)
print(tst.shape)
print(label.shape)

for col in tst.columns:
    if col != 'counts' and col != 'days' and col != 'Number_of_Dependants':
        #print(col)
        #print(tst[col][0:10])
        #print(ref[ref['var_name'] == col].coding)
        X=pd.get_dummies(tst[col], prefix=col)
        #print(X.shape)
        tst = tst.drop(col,axis=1)
        tst = pd.concat([tst, X], axis=1)
    else:
        print(col)
        tst[col] = tst[col].astype('int64')/max(tst[col].astype('int64'))
tst.head()

Index(['Marital_Status', 'Number_of_Dependents', 'Education_Level',
       'Currently_In_School', 'c6', 'Indigenous_Type', 'Status_Indian',
       'Refugee_Type', 'Citizenship_Status', 'Age', 'Lack_Housing',
       'Addiction', 'Visible_Minority', 'Child_at_Home', 'Single_Parent',
       'Physical_Disability', 'Learning_Disability',
       'Developmental_Disability', 'Visual_Disability', 'Hearing_Disability',
       'Mental_Health_Condition', 'Chronic_Illness', 'Short_term_Illness',
       'Other_Disability', 'sus', 'counts', 'days'],
      dtype='object')
Marital_Status                       object
Number_of_Dependents                float64
Education_Level                      object
Currently_In_School                  object
c6                                   object
Indigenous_Type                      object
Status_Indian                        object
Refugee_Type                         object
Citizenship_Status                   object
Age                                  obje

,counts,days,Marital_Status_I prefer not to say,Marital_Status_Married,Marital_Status_Single,Number_of_Dependents_0.0,Number_of_Dependents_1.0,Number_of_Dependents_2.0,Number_of_Dependents_3.0,Number_of_Dependents_4.0,...,Hearing_Disability_N,Hearing_Disability_Y,Mental_Health_Condition_N,Mental_Health_Condition_Y,Chronic_Illness_N,Chronic_Illness_Y,Short_term_Illness_N,Short_term_Illness_Y,Other_Disability_N,Other_Disability_Y
0,0.25,0.002445,0,0,1,1,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
1,0.25,0.000000,1,0,0,1,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
2,0.25,0.000000,1,0,0,1,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
3,0.25,0.000000,1,0,0,1,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
4,0.25,0.000000,1,0,0,1,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0


In [89]:
from sklearn.preprocessing import normalize
#print(tst['Marital_Status'].value_counts())
#print(tst.isnull().sum())
tst = normalize(tst, axis=1, norm='max')
print(tst)

[[0.25       0.00244466 0.         ... 0.         1.         0.        ]
 [0.25       0.         1.         ... 0.         1.         0.        ]
 [0.25       0.         1.         ... 0.         1.         0.        ]
 ...
 [0.25       0.02101811 0.         ... 0.         1.         0.        ]
 [0.25       0.00409928 0.         ... 0.         1.         0.        ]
 [0.25       0.         0.         ... 0.         1.         0.        ]]


In [90]:
print(tst.shape)
print(label.shape)

(69316, 81)
(69316,)


In [95]:
#pd.DataFrame(tst).to_csv('preprocessing/preprocessed_data_01_12_21.csv')

tst = pd.DataFrame(tst)
for col in tst.columns:
    if len(tst[col].unique()) == 1:
        tst.drop(col,inplace=True,axis=1)

features = tst
features = pd.DataFrame(features).reset_index()
print(label.value_counts())

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, label)
from sklearn.preprocessing import StandardScaler
import numpy as np
x_train = pd.DataFrame(x_train).fillna(value=0)
x_test = pd.DataFrame(x_test).fillna(value=0)
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
print(x_train[:2])
x_test = scaler.transform(x_test)

from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.90)
pca.fit(x_train)
print(pca.n_components_)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)
model = KNeighborsClassifier(n_neighbors=9)



# Train the model using the training sets
model.fit(x_train_pca,y_train)

1.0    40128
0.0    29188
Name: sus, dtype: int64
[[ 0.82077853 -0.27376691 -0.33084761 -0.72422424 -0.58339932  1.21959676
  -1.9870368  -0.32080363  3.49352607 -0.15489034 -0.08315398 -0.04541422
  -0.02149109 -0.01160462 -0.00877201 -0.00438588 -0.01754605  2.36480771
  -0.7387246  -0.15599364 -0.23827551 -0.4042729  -0.2908202  -0.2963473
  -0.26598663 -0.13310377 -0.18987274  0.71738437 -0.66293663 -0.18894647
  -0.08315398 -0.16848327 -0.2333214  -0.02809415 -0.1443404   0.28026075
   0.21417001 -0.21417001 -0.0227954  -0.10387948 -0.08017018  0.15849847
  -0.08350506 -0.27653052 -0.37321746  0.49673981 -0.0681025   0.15741105
  -0.15741105 -0.69273979 -0.51963452  1.07670203 -0.68766739  0.7236379
  -0.15213348 -0.70548541  0.99353849 -0.44363188 -0.68133421 -0.9624451
   1.9870368  -2.01312633 -0.38588777  3.69325079  0.28836221 -0.28836221
   0.20204568 -0.20204568  0.11767057 -0.11767057  0.10177803 -0.10177803
   0.11503827 -0.11503827  0.32160531 -0.32160531  0.20302347 -0.

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')

In [96]:
y_pred = model.predict(x_test_pca)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[3126 4143]
 [2872 7188]]
              precision    recall  f1-score   support

         0.0       0.52      0.43      0.47      7269
         1.0       0.63      0.71      0.67     10060

    accuracy                           0.60     17329
   macro avg       0.58      0.57      0.57     17329
weighted avg       0.59      0.60      0.59     17329



In [99]:
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
params = {'n_estimators': 600,
          'max_depth': 12,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}

reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(x_train_pca, y_train)

mse = mean_squared_error(y_test, reg.predict(x_test_pca))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

y_pred = reg.predict(x_test_pca)


The mean squared error (MSE) on test set: 0.2346


In [100]:
print(classification_report(y_test, y_pred.round()))
print(confusion_matrix(y_test, y_pred.round()))

              precision    recall  f1-score   support

         0.0       0.55      0.37      0.44      7269
         1.0       0.63      0.78      0.70     10060

    accuracy                           0.61     17329
   macro avg       0.59      0.58      0.57     17329
weighted avg       0.60      0.61      0.59     17329

[[2700 4569]
 [2193 7867]]


In [101]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-4,hidden_layer_sizes=(10, 10), random_state=1, max_iter = 2000)
clf.fit(x_train_pca, y_train)
y_pred = clf.predict(x_test_pca)
print(classification_report(y_test, y_pred.round()))
print(confusion_matrix(y_test, y_pred.round()))

              precision    recall  f1-score   support

         0.0       0.58      0.35      0.43      7269
         1.0       0.63      0.82      0.71     10060

    accuracy                           0.62     17329
   macro avg       0.60      0.58      0.57     17329
weighted avg       0.61      0.62      0.60     17329

[[2518 4751]
 [1857 8203]]


In [102]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-4,hidden_layer_sizes=(15, 15), random_state=1, max_iter = 5000)
clf.fit(x_train_pca, y_train)
y_pred = clf.predict(x_test_pca)
print(classification_report(y_test, y_pred.round()))
print(confusion_matrix(y_test, y_pred.round()))

              precision    recall  f1-score   support

         0.0       0.57      0.37      0.45      7269
         1.0       0.64      0.80      0.71     10060

    accuracy                           0.62     17329
   macro avg       0.60      0.59      0.58     17329
weighted avg       0.61      0.62      0.60     17329

[[2719 4550]
 [2045 8015]]


NameError: name 'tst' is not defined